---
- ToC
- Intro
- Methodik
- EDA + Process Model
    - Process durations (min max / avg) + metrics dom vs int 
    - -> bottleneck
    - Top 3 Traces to Process modell -> confromance check
    - rejected applications
    - unexpected behaviour

- missing details
- ML
- KPis
- process improvement
---

### Current Topics in main:

- Comparison dom vs int
- each Top 5 Variants

statistical analysis

- Head + description of column names (dom/)
- EDA (dom)
    - add: view_events_per_time-graph
- BPMN (dom)
- Process Tree (dom)
- (petri net) (dom)
- DFG (dom)

Process duration

- avg duration of whole process (dom)
- avg duration of each activity (dom)
- total durations of each activity (dom)
- unexpected behaviour : saved by employee (dom)

bottleneck (dom)

- dfg graph with durations
- insights from dfg
- insights from disco
- additonal remarks → unexpexted behaviour

rejected applications

- amount of rejection total (dom)
- amount of rejections based on role (dom)
- boxplot rejquested amount of rejected delcarations by role (dom)
- boxplot rejquested amount of approved delcarations by role (dom)
- Distribution of requested amount of rejected vs approved cases (dom)
---

## Introduction


---
## Methodology

---
## Exploratory Data Analysis

### Import

### Setup of logfiles

### Description of Columns: Domestic vs. International

### Comparison of Roles: Domestic vs. International

### Comparison of Activities: Domestic vs. International

### Comparison of Variants: Domestic vs. International

### Event Count by Resource: Domestic vs. International

### System Events: Domestic vs. International

### Amount Destribution: Domestic vs. International

### Events per Time Graph: Domestic vs. International

---
## Process Model(s)
### Petri Net or DFG
### Start + End Activities
### Unexpected Behaviour (only one case of...)
### Recycled declarations

### Differences between international and domestic declarations

---
## Specific Questions

### How long do the different process instances take? Are there any notable patterns?
- Average durations of the whole process
- Average duration of each activity
- Total duration of each activity 
> (Put visualization of avg + total durations next to each other)
- Unexpected behaviour: e.g. "Saved by employee" (dom)

### Are there any bottlenecks? If yes, where? If yes, can you think of any reasons?
- dfg graph with durations
- insights from dfg
- insight from diso

### How many applications get rejected? Can you find any reasons?
- amount of rejection total
- amount of rejections based on role
- boxplot rejquested amount of rejected delcarations by role
- boxplot rejquested amount of approved delcarations by role
- Distribution of requested amount of rejected vs approved cases

### Are there any patterns that would suggest non-conformance?
- TBD

### Social Network / Handover Network
- TBD

---
### What is missing to provide more detailed insights into the processes?
- TBD

### Reflect on where supervised / unsupervised machine learning techniques could help to obtain further insights.
- TBD

### Can you think of useful KPIs for the processes?
- TBD

### Make specific recommendations for improving the processes.
- TBD

---
## Conclusion